## pitchfork_spotify_data
**Program:** 04_pitchfork_spotify_data.ipynb <br>
**Created by:** Chris Chan<br>
**Date:** Apr 08, 2021<br>
**Purpose:** <br>
. Given the list of jazz albums we have from pitchfork, obtain various information though spotify's API.
. Some additional resources:
https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
https://stackoverflow.com/questions/36237522/get-spotipy-album-id-from-album
https://www.kaggle.com/pavansanagapati/spotify-music-api-data-extraction-part1


In [1]:
# loop over track ids 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm
from time import sleep
import time
import pandas as pd
import numpy as np

client_credentials = SpotifyClientCredentials(client_id = '060cca5dfadc41c7b852628460f55449', client_secret = '99d63147a4df4c7993bbc5ef0283442e')
sp = spotipy.Spotify(client_credentials_manager = client_credentials)

# client_credentials = SpotifyClientCredentials(client_id = 'd5c667b1cb884faeb1a61f42d2a9222a', client_secret = '47d9721a506b433b94c7afb7de36e276')
# sp = spotipy.Spotify(client_credentials_manager = client_credentials)

### Read-in Wide file for artist list

In [67]:
df_wide = pd.read_csv('../data/df_emo_top_wide_nm.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2238,2239) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [68]:
df_wide_art = df_wide[['artist','album']]
df_wide_art.head()

,artist,album
0,Sam Gendel,Fresh Bread
1,Femi Kuti / Made Kuti,Legacy +
2,Archie Shepp / Jason Moran,Let My People Go
3,Patricia Brennan,Maquishti
4,Baldi/Gerycz Duo,After Commodore Perry Service Plaza


In [72]:
album_list=list(df_wide_art['album'])
artist_list=list(df_wide_art['artist'])

In [70]:
album_list

['Fresh Bread',
 'Legacy +',
 'Let My People Go',
 'Maquishti',
 'After Commodore Perry Service Plaza',
 'Second Wave',
 'Good Guess',
 'Pardon My French',
 'Pieces of a Man',
 'Fuubutsushi',
 'Force Majeure',
 'Dimensional Stardust',
 'Self Help',
 'It Serve You Right to Suffer',
 'Share the Wealth',
 'LIVE',
 'Seven Storey Mountain VI',
 'America the Beautiful',
 'Sen Morimoto',
 'UPRIZE!',
 'Innercity Griots',
 'Palo Alto',
 'So Glossy and So Thin',
 'Fly Moon Die Soon',
 'The Fire Still Burns',
 'Mama, You Can Bet!',
 'Source',
 'Actual Presence',
 'Manger on McNichols',
 'Wu Hen',
 'African Rhythms 1970-1982',
 'Dialectic Soul',
 'Symphonie Pacifique',
 'New Grass',
 'The Shadows and the Light',
 'Roy Ayers JID 002',
 'on the tender spot of every calloused moment',
 'Shades of Flu: Healthy Remixes For an Ill Moment',
 'Becoming (Music From the Netflix Original Documentary)',
 'Yeo\u200b-\u200bNeun',
 'Attica Blues',
 'Eastern Flowers',
 'What Kinda Music',
 'My Garden',
 'Live in 

**To get album URI**

In [43]:
# just an example if manual

# name = ["Fresh Bread"]
# result = sp.search(name,type="album") 
# result['albums']['items'][0]['uri']

'spotify:album:3iVOk8jehk1yzUvfA9iHvC'

In [79]:
# test list
album_list2=['Palo Alto','So Glossy and So Thin']

In [83]:
album_uris =[]
album_names=[]

for i in album_list:
    try:
        result = sp.search(i,type="album")
        album_uris.append(result['albums']['items'][0]['uri'])
        album_names.append(i)
    except:
        result=np.nan
        album_uris.append(result)
        album_names.append(i)

#album_uris

**Convert Album and Artist List to DF**

In [98]:
df_album_uris=pd.DataFrame(album_uris,columns=['album_uri'])
df_album_names=pd.DataFrame(album_names,columns=['album_names'])

In [99]:
df_album_uris.head()

,album_uri
0,spotify:album:3iVOk8jehk1yzUvfA9iHvC
1,spotify:album:6WgxIWDCnLtIaDYHHsJCIh
2,spotify:album:3nD9Oab8M4sCLNmqmwSOdA
3,spotify:album:52xnMW8ir7yfyuVzUSpeTZ
4,spotify:album:7uxuFsYZuMU4rkuXQzlLHf


**Create embed link and join back to DF**

In [88]:
import regex as re

In [91]:
type(album_uris)

list

In [100]:
# new data frame with split value columns
new = df_album_uris["album_uri"].str.split(":", n = 2, expand = True)
  
# making separate first name column from new data frame
df_album_uris["album_uri_new"]= new[2]
  
# Dropping old Name columns
# data.drop(columns =["Name"], inplace = True)
  
# df display

In [101]:
df_album_uris.head()

,album_uri,album_uri_new
0,spotify:album:3iVOk8jehk1yzUvfA9iHvC,3iVOk8jehk1yzUvfA9iHvC
1,spotify:album:6WgxIWDCnLtIaDYHHsJCIh,6WgxIWDCnLtIaDYHHsJCIh
2,spotify:album:3nD9Oab8M4sCLNmqmwSOdA,3nD9Oab8M4sCLNmqmwSOdA
3,spotify:album:52xnMW8ir7yfyuVzUSpeTZ,52xnMW8ir7yfyuVzUSpeTZ
4,spotify:album:7uxuFsYZuMU4rkuXQzlLHf,7uxuFsYZuMU4rkuXQzlLHf


**create link to embed music album**

In [112]:
df_album_uris["album_uri_link"] = ("https://open.spotify.com/embed/album/" + df_album_uris['album_uri_new'])
df_album_uris.head()

,album_uri,album_uri_new,album_uri_link
0,spotify:album:3iVOk8jehk1yzUvfA9iHvC,3iVOk8jehk1yzUvfA9iHvC,https://open.spotify.com/embed/album/3iVOk8jeh...
1,spotify:album:6WgxIWDCnLtIaDYHHsJCIh,6WgxIWDCnLtIaDYHHsJCIh,https://open.spotify.com/embed/album/6WgxIWDCn...
2,spotify:album:3nD9Oab8M4sCLNmqmwSOdA,3nD9Oab8M4sCLNmqmwSOdA,https://open.spotify.com/embed/album/3nD9Oab8M...
3,spotify:album:52xnMW8ir7yfyuVzUSpeTZ,52xnMW8ir7yfyuVzUSpeTZ,https://open.spotify.com/embed/album/52xnMW8ir...
4,spotify:album:7uxuFsYZuMU4rkuXQzlLHf,7uxuFsYZuMU4rkuXQzlLHf,https://open.spotify.com/embed/album/7uxuFsYZu...


**merge back to file and output csv**

In [113]:
df_wide2 = pd.concat([df_wide,df_album_uris],axis=1)
df_wide2.head()

,Unnamed: 0,artist,album,score,genre,review,othercat2,best,review_clean,othercat_clean,...,instrumentalness,liveness,valence,tempo,img_link,audio_link,audio_link2,album_uri,album_uri_new,album_uri_link
0,0,Sam Gendel,Fresh Bread,6.5,Jazz,"Fifty-two tracks deep, this archival compendiu...","Sam Gendel, Jay Balfour, Sam Gendel, cover...",0,fiftytwo tracks deep this archival compendium ...,"sam gendel, jay balfour, sam gendel, cover...",...,0.394020,0.205935,0.453084,114.752799,https://media.pitchfork.com/photos/601c69fcfdb...,NaN,../img/Sam Gendel - Afro Blue.mp3,spotify:album:3iVOk8jehk1yzUvfA9iHvC,3iVOk8jehk1yzUvfA9iHvC,https://open.spotify.com/embed/album/3iVOk8jeh...
1,1,Femi Kuti / Made Kuti,Legacy +,7.8,Global,"On the duo’s generation-spanning double album,...","Femi Kuti, Made Kuti, Joey Akan, Global, F...",0,on the duo’s generationspanning double album f...,"femi kuti, made kuti, joey akan, global, f...",...,0.252181,0.212943,0.428999,112.966493,https://media.pitchfork.com/photos/602ed922e06...,NaN,../img/Femi Kuti - Day By Day.mp3,spotify:album:6WgxIWDCnLtIaDYHHsJCIh,6WgxIWDCnLtIaDYHHsJCIh,https://open.spotify.com/embed/album/6WgxIWDCn...
2,2,Archie Shepp / Jason Moran,Let My People Go,7.5,Jazz,These live sets between the saxophonist and pi...,"Archie Shepp, Jason Moran, Madison Bloom, ...",0,these live sets between the saxophonist and pi...,"archie shepp, jason moran, madison bloom, ...",...,0.252181,0.212943,0.428999,112.966493,https://media.pitchfork.com/photos/6026aecd8bf...,NaN,../img/Archie Shepp - Blue And Sentimental.mp3,spotify:album:3nD9Oab8M4sCLNmqmwSOdA,3nD9Oab8M4sCLNmqmwSOdA,https://open.spotify.com/embed/album/3nD9Oab8M...
3,3,Patricia Brennan,Maquishti,7.4,Experimental,"Challenging but never chaotic, contemplative y...","Patricia Brennan, Daniel Felsenthal, Experime...",0,challenging but never chaotic contemplative ye...,"patricia brennan, daniel felsenthal, experime...",...,0.252181,0.212943,0.428999,112.966493,https://media.pitchfork.com/photos/60008fcbf70...,NaN,../img/patricia-brennan-sonnet.mp3,spotify:album:52xnMW8ir7yfyuVzUSpeTZ,52xnMW8ir7yfyuVzUSpeTZ,https://open.spotify.com/embed/album/52xnMW8ir...
4,4,Baldi/Gerycz Duo,After Commodore Perry Service Plaza,7.1,Jazz,The second album of improvisational jazz from ...,"Baldi/Gerycz Duo, Fred Thomas, Experimental,...",0,the second album of improvisational jazz from ...,"baldi/gerycz duo, fred thomas, experimental,...",...,0.252181,0.212943,0.428999,112.966493,https://media.pitchfork.com/photos/5ff4c5e0f17...,NaN,"../img/Baldi, Gerycz - Phonetic Tramsway, Pt. ...",spotify:album:7uxuFsYZuMU4rkuXQzlLHf,7uxuFsYZuMU4rkuXQzlLHf,https://open.spotify.com/embed/album/7uxuFsYZu...


In [114]:
df_wide2.shape

(624, 2243)

In [115]:
df_wide2.to_csv(r'../data/df_emo_top_wide_nm_spuri.csv')

**To get all albums per artist**

In [44]:
name = ["Micheal Jackson","pitbull","Christina","Elvis Presley"]
result = sp.search(name) 
result['tracks']['items'][1]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/27XaM1Qqd3zfxJ82cbQ1FK'},
  'href': 'https://api.spotify.com/v1/artists/27XaM1Qqd3zfxJ82cbQ1FK',
  'id': '27XaM1Qqd3zfxJ82cbQ1FK',
  'name': 'Micheal Jackson For Siegfried',
  'type': 'artist',
  'uri': 'spotify:artist:27XaM1Qqd3zfxJ82cbQ1FK'}]

In [45]:
#Extract Artist's uri
artists_uris = result['tracks']['items'][0]['artists'][0]['uri']
#Pull all of the artist's albums
artist_albums = sp.artist_albums(artists_uris, album_type='album')
#Store artist's albums' names' and uris in separate lists
artist_album_names = []
artist_album_uris = []
for i in range(len(artist_albums['items'])):
    artist_album_names.append(artist_albums['items'][i]['name'])
    artist_album_uris.append(artist_albums['items'][i]['uri'])
    
artist_album_names
artist_album_uris
#Keep names and uris in same order to keep track of duplicate albums


['spotify:album:2X8UOIkZQdcz2Hi5Ynt2uk',
 'spotify:album:7pomP86PUhoJpY3fsC0WDQ',
 'spotify:album:54WFfZzds26NrwcmStfamd',
 'spotify:album:0yYWeLqonFk11pmb0RiMbP',
 'spotify:album:52E4RP7XDzalpIrOgSTgiQ',
 'spotify:album:0RNsFWWdiz1rrdLI1pwbvJ',
 'spotify:album:6BYNnU3IYdmYCNHtvlFngH',
 'spotify:album:3T1JYXVBJiGCyTpeGk6ekt',
 'spotify:album:3OBhnTLrvkoEEETjFA3Qfk',
 'spotify:album:3Y4ZambRFbDzlnCtqEUrgY',
 'spotify:album:0oX4SealMgNXrvRDhqqOKg',
 'spotify:album:3Us57CjssWnHjTUIXBuIeH',
 'spotify:album:24TAupSNVWSAHL0R7n71vm',
 'spotify:album:5Ne1SPpiXQqpCZ6pZCRfIM',
 'spotify:album:2ANVost0y2y52ema1E9xAZ',
 'spotify:album:1C2h7mLntPSeVYciMRTF4a',
 'spotify:album:2ZytN2cY4Zjrr9ukb2rqTP',
 'spotify:album:3dZ91ooVpNEdukDWuNzbpZ',
 'spotify:album:6leeBdAohG7zhybC3Cmu8W',
 'spotify:album:3RNMz43pauYka09DdYHpHZ']

In [46]:
artist_album_names

['Scream',
 'XSCAPE',
 'XSCAPE - Track by Track Commentary',
 'Michael',
 'Invincible',
 'BLOOD ON THE DANCE FLOOR/ HIStory In The Mix',
 'BLOOD ON THE DANCE FLOOR/ HIStory In The Mix',
 'Manila 1996',
 'HIStory - PAST, PRESENT AND FUTURE - BOOK I',
 'HIStory - PAST, PRESENT AND FUTURE - BOOK I',
 'Dangerous',
 'Bad (Remastered)',
 'Bad 25th Anniversary',
 'Thriller',
 'Thriller',
 'Thriller 25 Super Deluxe Edition',
 'Off the Wall',
 'Forever Michael',
 'Forever, Michael',
 'Music And Me']

In [ ]:
import spotipy

sp = spotipy.Spotify()
sp.trace = False

# find album by name
album = "Pestbringer"
results = sp.search(q = "album:" + album, type = "album")

# get the first album uri
album_id = results['albums']['items'][0]['uri']

# get album tracks
tracks = sp.album_tracks(album_id)
for track in tracks['items']:
    print(track['name'])